# Lot sizing

<div class="alert alert-block alert-info">
    &#9432; The code in this notebook can be executed <a href="https://www.opvious.io/examples/retro/notebooks/?path=lot-sizing.ipynb">directly from your browser</a>.
</div>

This notebook implements a dynamic lot sizing model, as described in [this blog post](
https://towardsdatascience.com/the-dynamic-lot-size-model-a-mixed-integer-programming-approach-4a9440ba124e).

In [1]:
%pip install 'opvious>=0.16.8'

## Model

We start by defining our MIP model using `opvious`' [declarative modeling API](https://opvious.readthedocs.io/en/stable/modeling.html).

In [2]:
import opvious.modeling as om

class LotSizing(om.Model):
    """Lot sizing MIP model"""
    
    # Inputs
    horizon = om.Parameter.natural()  # Number of days in schedule
    days = om.interval(0, horizon()-1, name="T")
    holding_cost = om.Parameter.non_negative(days)  # Marginal cost of storing inventory
    setup_cost = om.Parameter.non_negative(days)  # Fixed cost of producing in a given day
    demand = om.Parameter.non_negative(days)  # Demand per day
    
    # Outputs
    production = om.Variable.non_negative(days, upper_bound=demand.total())  # Production per day
    is_producing = om.fragments.ActivationVariable(production)  # 1 if production > 0
    inventory = om.Variable.non_negative(days)  # Stored inventory per day
    
    @om.objective
    def minimize_cost(self):
        return om.total(
            self.holding_cost(d) * self.inventory(d) + self.setup_cost(d) * self.is_producing(d)
            for d in self.days
        )
    
    @om.constraint
    def inventory_propagation(self):
        for d in self.days:
            base = om.switch((d > 0, self.inventory(d-1)), 0)  # Previous day inventory (0 initially)
            delta = self.production(d) - self.demand(d)
            yield self.inventory(d) == delta + base


model = LotSizing()
model.specification()

<div style="margin-top: 1em; margin-bottom: 1em;">
<details open>
<summary style="cursor: pointer; text-decoration: underline; text-decoration-style: dotted;">LotSizing</summary>
<div style="margin-top: 1em;">
$$
\begin{align*}
  \S^p_\mathrm{horizon}&: h \in \mathbb{N} \\
  \S^a&: T \doteq \{ 0 \ldots h - 1 \} \\
  \S^p_\mathrm{holdingCost}&: c^\mathrm{holding} \in \mathbb{R}_+^{T} \\
  \S^p_\mathrm{setupCost}&: c^\mathrm{setup} \in \mathbb{R}_+^{T} \\
  \S^p_\mathrm{demand}&: d \in \mathbb{R}_+^{T} \\
  \S^v_\mathrm{production}&: \pi \in [0, \sum_{t \in T} d_{t}]^{T} \\
  \S^v_\mathrm{isProducing}&: \pi^\mathrm{is} \in \{0, 1\}^{T} \\
  \S^c_\mathrm{isProducingActivates}&: \forall t \in T, \sum_{t' \in T} d_{t'} \pi^\mathrm{is}_{t} \geq \pi_{t} \\
  \S^v_\mathrm{inventory}&: \iota \in \mathbb{R}_+^{T} \\
  \S^o_\mathrm{minimizeCost}&: \min \sum_{t \in T} \left(c^\mathrm{holding}_{t} \iota_{t} + c^\mathrm{setup}_{t} \pi^\mathrm{is}_{t}\right) \\
  \S^c_\mathrm{inventoryPropagation}&: \forall t \in T, \iota_{t} = \pi_{t} - d_{t} + \begin{cases} \iota_{t - 1} \mid t > 0, \\ 0 \end{cases} \\
\end{align*}
$$
</div>
</details>
</div>

## Application

We use the above model to write a function which will return an optimal schedule.

In [3]:
import logging
import opvious

logging.basicConfig(level=logging.INFO)  # Display live progress notifications

async def optimal_production(inputs):
    """Computes an optimal production schedule for the given inputs"""
    problem = opvious.Problem(
        specification=model.specification(),
        parameters={
            'demand': inputs_df['demand'],
            'holdingCost': inputs_df['inventory_cost'],
            'setupCost': inputs_df['setup_cost'],
            'horizon': len(inputs_df),
        },
    )
    solution = await opvious.Client.default().solve(problem)
    return solution.outputs.variable('production').sort_index()

## Example

Let's now introduce some sample data, identical to the [original example's](https://raw.githubusercontent.com/bruscalia/optimization-demo-files/main/mip/dynamic_lot_size/data/input_wagner.csv). 

In [4]:
import io
import pandas as pd

inputs_df = pd.read_csv(io.StringIO("""
id,setup_cost,inventory_cost,demand
1,85,1.0,69
2,102,1.0,29
3,102,1.0,36
4,101,1.0,61
5,98,1.0,61
6,114,1.0,26
7,105,1.0,34
8,86,1.0,67
9,119,1.0,45
10,110,1.0,67
11,98,1.0,79
12,114,1.0,56
""")).set_index('id')

inputs_df

,setup_cost,inventory_cost,demand
id,,,
1,85,1.0,69
2,102,1.0,29
3,102,1.0,36
4,101,1.0,61
5,98,1.0,61
6,114,1.0,26
7,105,1.0,34
8,86,1.0,67
9,119,1.0,45


In [5]:
await optimal_production(inputs_df)

INFO:opvious.client.handlers:Validated inputs. [parameters=37]
INFO:opvious.client.handlers:Solving problem... [columns=39, rows=26]
INFO:opvious.client.handlers:Solve in progress... [iterations=0, gap=inf]
INFO:opvious.client.handlers:Solve in progress... [iterations=0, gap=100.0%]
INFO:opvious.client.handlers:Solve in progress... [iterations=12, gap=92.0%]
INFO:opvious.client.handlers:Solve in progress... [iterations=23, gap=38.93%]
INFO:opvious.client.handlers:Solve in progress... [iterations=27, gap=25.94%]
INFO:opvious.client.handlers:Solve completed with status OPTIMAL. [objective=779]


,value
0,98
3,97
5,121
8,112
10,67
11,135
